In [1]:
import requests
import base64
import csv
import time

# ========== Step 1: Spotify Access Token ==========
def get_access_token(client_id, client_secret):
    auth = f"{client_id}:{client_secret}"
    b64_auth = base64.b64encode(auth.encode()).decode()

    token_url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": f"Basic {b64_auth}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }

    response = requests.post(token_url, headers=headers, data=data)
    response.raise_for_status()
    return response.json().get("access_token")


# ========== Step 2: Search for Playlist ==========
def search_playlist(token, year):
    url = "https://api.spotify.com/v1/search"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "q": f"top hits {year}",
        "type": "playlist",
        "limit": 1
    }

    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    playlists = response.json().get("playlists", {}).get("items", [])
    return playlists[0] if playlists else None


# ========== Step 3: Get Tracks from Playlist ==========
def get_playlist_tracks(token, playlist_id):
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
    headers = {"Authorization": f"Bearer {token}"}
    all_tracks = []

    while url:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        for item in data['items']:
            track = item.get('track')
            if not track:
                continue
            name = track.get('name')
            artists = ", ".join([a['name'] for a in track.get('artists', [])])
            link = track.get('external_urls', {}).get('spotify')
            all_tracks.append((name, artists, link))
        url = data.get('next')  # for pagination
        time.sleep(0.1)  # avoid hitting rate limit
    return all_tracks


# ========== Step 4: Main Program ==========
client_id = "ca8d23e91c544e949390f428f95924db"
client_secret = "fc2e726f8b50426cbdd785959b40ef98"

token = get_access_token(client_id, client_secret)
filename = "spotify_top_hits_2011_2025.csv"

with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Year", "Playlist Name", "Track Name", "Artist(s)", "Spotify URL"])

    for year in range(2011, 2026):
        try:
            playlist = search_playlist(token, year)
            if not playlist:
                print(f"{year} - No playlist found.")
                continue

            playlist_id = playlist['id']
            playlist_name = playlist['name']
            print(f"Fetching: {playlist_name} ({year})")

            tracks = get_playlist_tracks(token, playlist_id)
            for name, artist, url in tracks:
                writer.writerow([year, playlist_name, name, artist, url])

        except Exception as e:
            print(f"{year} - Error: {e}")

print(f"\n✅ CSV saved as: {filename}")
import os

filename = "spotify_top_hits_2011_2025.csv"

if os.path.exists(filename):
    print("✅ File saved successfully at:", os.path.abspath(filename))
else:
    print("❌ File not found. Please check the code or path.")



Fetching: Top Hits of 2011 (2011)
Fetching: Top Hits of 2012 (2012)
3 (2013)

2014 - No playlist found.
Fetching: Top Hits of 2015 (2015)
6 (2016)

Fetching: TOP HITS OF 2017 (2017)
op Hits of 2018 (2018)

2019 - No playlist found.
Fetching: Top Hits of 2020 (2020)
1 (2021)

Fetching: Top Hits of 2022 (2022)
3 (2023)

Fetching: Top Hits of 2024 (2024)
Bollywood Hits 2025 (2025)

✅ CSV saved as: spotify_top_hits_2011_2025.csv
✅ File saved successfully at: C:\Users\BAPS\spotify_top_hits_2011_2025.csv
